<a href="https://colab.research.google.com/github/FloxTBoTyy/Mi-primer-repository/blob/main/dashboard_spacex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# Import required libraries
import pandas as pd
import dash
#from jupyter_dash import JupyterDash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
url= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
# to callback pie
spacex_filtrado_total=spacex_df[['Launch Site','class']].groupby('Launch Site',as_index=False).sum()
count_class=spacex_df[['Launch Site','class']].groupby('Launch Site',as_index=False).count()['class']

sum_class = spacex_filtrado_total['class']
sum_launch = spacex_filtrado_total['Launch Site']

ceros =count_class - sum_class
transpose_data=pd.DataFrame({'Launch Site':sum_launch, 'unos':sum_class, 'ceros':ceros}).set_index('Launch Site').transpose()
# Create a dash application
app = dash.Dash(__name__)
#app = JupyterDash(__name__)
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                      {'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'CCAFS SLC-40	', 'value': 'CCAFS SLC-40'},
                                                      {'label': 'KSC LC-39A	', 'value': 'KSC LC-39A'},
                                                      {'label':'VAFB SLC-4E	', 'value': 'VAFB SLC-4E'}
                                                                                            ],
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True
                                                            ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000:'5000',
                                                       7500:'7500',
                                                       10000:'10000'},
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):

    if entered_site == 'ALL':
        fig = px.pie(spacex_filtrado_total, values='class',
        names=spacex_filtrado_total['Launch Site'],
        title='Total success launches')
        return fig
    else:
        fig = px.pie(transpose_data[entered_site], values=entered_site,
        names=transpose_data.index,
        title='Total success launches for site '+ str(entered_site))
        return fig
        # return the outcomes piechart for a selected site
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")]
)

def get_scatter_chart(site_value,payload_value):
  if site_value== 'ALL':
    fig = px.scatter(spacex_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',range_x=[payload_value[0], payload_value[1]],
                     title='Correlation between Payload and Success for ALL sites')
    return fig

  else:
    fig = px.scatter(spacex_df[spacex_df['Launch Site']==site_value],x='Payload Mass (kg)',y='class',color='Booster Version Category',range_x=[payload_value[0], payload_value[1]]
                               ,title='Correlation between Payload and Success for '+ str(site_value))
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
    #app.run_server(mode='inline')


<IPython.core.display.Javascript object>